# その他の情報

基本演習の最後に、ここまで解説してこなかったいくつかの機能について解説していきます。

## ループ処理

Playbook を記述する際に、いくつかのループ処理を用いると簡潔な記述が可能となります。

例

```
- yum:
    name: httpd
    state: latest

- yum:
    name: vim
    state: latest

- yum:
    name: emacs
    state: latest
```

というPlaybookは、

```
- yum:
    name: "{{ item }}"
    state: latest
  with_items:
    - httpd
    - vim
    - emacs
```

と記述することができます。

この他にも様々なケースで使えるループがありますので、どのようなものがあるか眺めてみてください。

* [Loops](http://docs.ansible.com/ansible/latest/playbooks_loops.html)

## include, import

Ansible にはPlaybookや、分解してTaskのみを記述したファイルを別のPlaybookやTaskから読み込むことができます。

以下のような2つのYAMLを準備しました。

* [import_include_playbook.yml](/edit/aitac-automation-handson/import_include_playbook.yml) ・・・単体で動作するPlaybook
* [import_include_task.yml](/edit/aitac-automation-handson/import_include_task.yml) ・・・ Task のみを抜粋したTaskファイル

これらの機能は、一度作ったPlaybookを再利用する場合に重要となります。

### import_playbook

まず、import_playbookを使って、外部のPlaybookを呼び出してみます。

* [import_include_example_playbook.yml](/edit/aitac-automation-handson/import_include_example_playbook.yml)

In [ ]:
# この Cell を実行

ansible-playbook import_include_example_playbook.yml

このように、import_playbookを使うと、複数のPlaybookを連続して実行するこが可能となります。また、例で実施しているように、実行時に変数の値を書き換えることも可能です。

### import/include_tasks

次に、import_tasks, include_tasks を使って、外部のTaskファイルを呼び出してみます。

* [import_include_example_task.yml](/edit/aitac-automation-handson/import_include_example_task.yml)

In [ ]:
# この Cell を実行

ansible-playbook import_include_example_task.yml

このように、外部に配置したTaskファイルを読み込むことが可能です。

import_tasks, include_tasks の違いは、どのタイミングでTaskファイルが読み込まれるかが違います。

* import_tasks ・・・Playbookの実行前にファイルが読み込まれます（先読み、静的）
* include_tasks ・・・Playbookが実行され、該当のinclude箇所まで来た時にファイルが読み込まれます（後読み、動的）

ループや変数と組み合わせて動的に読み込むTaskファイルを変える場合には include_tasks を使います。
それ以外は import_tasks を使っておけばほぼ問題はありません。

これらの詳細な違いについては以下のドキュメントに詳細が解説されています。

* [Creating Reusable Playbooks](https://docs.ansible.com/ansible/2.4/playbooks_reuse.html)
* [Including and Importing](https://docs.ansible.com/ansible/2.4/playbooks_reuse_includes.html)





## role によるPlaybook管理

Playbook は 先の include/import を使うことで部品化と再利用が行えるようになりますが、Ansible では更に Playbook を汎用化して再利用を容易にする仕組みを持っており、それが role になります。

* [roles](http://docs.ansible.com/ansible/latest/playbooks_reuse_roles.html)

role を使うには Ansible を実行する際のカレントディレクトリに、roles をというディレクトリを作成し、その配下に決まった形式でファイルを配置します。

以下はサンプルのロール「sample_ping」のディレクトリ構成です。

* tasks ・・・実際に処理を行うTaskを記載
* defaults ・・・デフォルトの変数と値を定義
* vars ・・・環境によって異なる変数を定義
* files ・・・ copy モジュール等で配布されるファイルを配置する
* templates ・・・ template モジュールで配布される j2 テンプレートファイルを配置する
* meta ・・・このRoleの挙動を制御するメタデータを設定する
* handler ・・・notifyを受け取って動作する handler を定義。handler は Task の中の changed をトリガーに実行される Task です
* tests ・・・ role のユニットテストを記述する

これら全てを使う必要はなく、以下ので例では、tasks, defaults のみを使ってroleを定義していています。

In [1]:
tree roles/sample_ping

roles/sample_ping
├── defaults
│   └── main.yml
├── files
├── handlers
│   └── main.yml
├── meta
│   └── main.yml
├── README.md
├── tasks
│   └── main.yml
├── templates
├── tests
│   ├── inventory
│   └── test.yml
└── vars
    └── main.yml

8 directories, 8 files


上記では以下の2ファイルを編集しています。

* [tasks/main.yml](/edit/aitac-automation-handson/roles/sample_ping/tasks/main.yml) ・・・ Taskを記載
* [defaults/main.yml](/edit/aitac-automation-handson/roles/sample_ping/defaults/main.yml) ・・・デフォルトの変数値を記載

この Role を利用する [sample_role.yml](http://13.112.122.31:8888/edit/aitac-automation-handson/sample_role.yml) を実行してみましょう。

In [ ]:
# この Cell を実行

ansible-playbook sample_role.yml

Playbook から ロールを指定して呼び出すことで、何度でも同じ処理を実行することが可能となります。また、必要に応じて変数の上書きなども可能です。

role を作成するには、ansible-galaxy コマンドを使うと雛形を簡単に作成することができます。以下のコマンドでは roles ディレクトリ以下に、role名 `role_name` を作成してくれます。

In [ ]:
# 実行例

ansible-galaxy init roles/role_name

## ansible galaxy

Ansible では上記の role を使う事で再利用性の高い Playbook を記述できるようになります。

世界中で開発された role が登録され、自由に使うことができる [Ansible Galaxy](https://galaxy.ansible.com) というサイトもあります。

このサイトに登録された role はそのまま使うこともできますし、自分のやりたい処理の参考にすることも可能です。

## その他の情報

* [Ansible公式ドキュメント](https://docs.ansible.com)
* [Intro to Playbooks](http://docs.ansible.com/ansible/latest/playbooks_intro.html)

## 自由演習（時間が余った方向け）

aitac-automation-handson/scratch ディレクトリに好きなPlaybookを作成して自由に実行し、これまでの内容を復習してください。

新しいPlaybookを作成する際のテンプレートは以下になります。

* [scratch/example.yml](/edit/aitac-automation-handson/scratch/example.yml)

In [ ]:
# 実行例

ansible-playbook -i inventories/01_aws_ops scratch/example.yml